In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-07-14 12:58:46--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-14 12:58:47 (25.8 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [11]:
import minsearch
from minsearch import Index
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [9]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [13]:
index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)


In [ ]:
select * from where course = 'data-engineering-zoomcamp';

In [15]:
q = 'the course has already started, can i still enroll?'

In [14]:
index.fit(documents)

In [19]:
boost = {'question': 3.0, 'section':0.5}
results=index.search(query = q,
                     filter_dict={'course':'data-engineering-zoomcamp'},
             boost_dict=boost,
             num_results=5)
            

In [49]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [30]:
from openai import OpenAI


In [36]:
client = OpenAI(api_key='You OPENAI API KEY here')

In [66]:
response= client.chat.completions.create(model='gpt-4o',
                               messages=[{"role": "user", "content": q}])
response.choices[0].message.content

"It's possible that you might still be able to enroll in a course even if it has already started, but it largely depends on several factors:\n\n1. **Institution Policy:** Different universities and educational institutions have their own policies regarding late enrollment. Some may have specific deadlines or windows during which late enrollment is allowed.\n\n2. **Course Structure:** Some courses, especially those that are more flexible or self-paced (such as many online courses), might allow you to join even after the official start date. Traditional, in-person courses may be less flexible.\n\n3. **Instructor Discretion:** In some cases, the decision may be left up to the instructor of the course. You could reach out to them directly to explain your situation and see if they are willing to let you join late.\n\n4. **Catch Up:** You should consider how much material you have missed and whether you will realistically be able to catch up. Some courses have materials that build on each ot

"It depends on the specific policies of the course or institution offering it. Many schools and online course platforms have policies about late enrollment. Here are a few steps you can take:\n\n1. **Check the Course Website or Syllabus:** Look for any information regarding late enrollment or add/drop deadlines.\n2. **Contact the Instructor:** Reach out to the course instructor or professor and explain your situation. They may allow you to join late, especially if you have a valid reason.\n3. **Contact the Academic Advisor or Registrar:** If you’re enrolled in a degree program, your academic advisor or the registrar's office may be able to provide guidance on enrolling late.\n4. **Self-Study:** If enrolling late isn’t an option, you might be able to catch up on your own and join the next session of the course.\n5. **Online Courses:** Platforms like Coursera, edX, or Udemy often offer flexible enrollment options, allowing you to start courses at any time.\n\nBeing proactive and demonstr

In [61]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION on the CONTEXT from the FAQ database. 
use only the facts from the CONTEXT when answering the QUESTION.
if the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}


CONTEXT: {context}
""".strip()
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n "

In [58]:
context

"section: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\n section: General course-related questions\nquestion: Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.\n\n section: General course-related questions\nquestion: Course - When will the course start?\nanswer: The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The cours

In [64]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION on the CONTEXT from the FAQ database. 
use only the facts from the CONTEXT when answering the QUESTION.
if the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can i still enroll?


CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

 section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final 

In [67]:
response= client.chat.completions.create(model='gpt-4o',
                               messages=[{"role" : "user", "content": prompt}])
response.choices[0].message.content

"Yes, even if the course has already started, you're still eligible to enroll and submit the homework. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

### Modularizing Code

In [91]:
def search(query):
    boost = {'question': 3.0, 'section':0.5}
    results=index.search(query = query,
            filter_dict={'course':'data-engineering-zoomcamp'},
             boost_dict=boost,
             num_results=10)

    return results

In [92]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION on the CONTEXT from the FAQ database. 
use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}


CONTEXT: {context}
""".strip()
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n "
    prompt= prompt_template.format(question=query,context=context).strip()
    return prompt

In [93]:
def llm(prompt):
    response= client.chat.completions.create(model='gpt-4o',
                               messages=[{"role": "user", "content": prompt}])
    return response.choices[0].message.content

In [95]:
query = 'how do i run kafka ?'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)
answer

'To run Kafka producer, consumer, kstreams, etc., using Java in the terminal, follow these steps:\n\n1. Navigate to your project directory.\n2. Run the following command in the terminal:\n\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of your jar file. This command assumes you already have your project compiled and the necessary jar file created.'